# NumPy Linear Algebra Lab  
### A Practical Math Engine for Machine Learning Systems

This repository is not a collection of NumPy examples.  
It is a **designed math layer** that implements, tests, and validates the core linear algebra operations used inside real machine learning and optimization pipelines.

The goal is simple:  
> Treat linear algebra as **software infrastructure**, not as theory.

---

## System Overview

The project is organized as a modular math engine where each layer represents a real engineering responsibility:



In [1]:
import numpy as np
from numpy import linalg  as LA

In [2]:
A  = np.array([[1 ,2 ,3] , [np.nan , 4,5], [0 , 8 , 9]])
A

array([[ 1.,  2.,  3.],
       [nan,  4.,  5.],
       [ 0.,  8.,  9.]])

In [3]:
A.shape

(3, 3)

In [4]:
A.ndim #Dimensions of the array

2

In [6]:
A.size #total numbers inside the array

9

In [7]:
print(np.isfinite(A))

[[ True  True  True]
 [False  True  True]
 [ True  True  True]]


## Basic Building Blocks

## `np.diag` - Create or extract diagonal

In [8]:
np.diag(A)

array([1., 4., 9.])

# `np.triu()` and  `np.tril()` - Extract Lower and Upper triangle of the matrix